In [ ]:
import sys
sys.path.insert(0, "src")

In [ ]:
from __future__ import annotations

from modeling import Transformer
from miscellaneous import get_conversion_rules, convert_weights, get_sharding_rules
from transformers import AutoTokenizer, AutoModelForCausalLM

import jax
import jax.numpy as jnp
import chex
from jax.sharding import Mesh, NamedSharding
from jax.experimental import mesh_utils
from jax.experimental.pjit import pjit

In [ ]:
mesh = Mesh(mesh_utils.create_device_mesh((1, 8)), ("dp", "mp"))
mesh

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("KRAFTON/KORani-v1-13B")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [ ]:
model_hf = AutoModelForCausalLM.from_pretrained("KRAFTON/KORani-v1-13B")

model = Transformer(
    vocab_size=model_hf.config.vocab_size,
    layers=model_hf.config.num_hidden_layers,
    dim=model_hf.config.hidden_size,
    heads=model_hf.config.num_attention_heads,
    hidden=model_hf.config.intermediate_size,
    rotary=int(model_hf.config.rotary_pct * model_hf.config.hidden_size // model_hf.config.num_attention_heads),
    eps=model_hf.config.layer_norm_eps,
)
params = jax.tree_map(
    lambda param, spec: jax.device_put(param, NamedSharding(mesh, spec)),
    convert_weights(model_hf.state_dict(), get_conversion_rules(model)),
    get_sharding_rules(model)
)

In [ ]:
text = """# Document
전년도 대비 79명 늘어 1019명, 행정수요 대처 광양시의 공무원 정원이 크게 늘어나 행정서비스 향상이 기대된다. 
시는 행정안전부에서 발표한 2018년도 자치단체 기준인건비에서 광양시의 일반직 정원이 지난해보다 79명이 늘어난 1019명으로 산정됐다고 밝혔다. 
지난 1995년 도농통합 당시 991명이었던 광양시 공무원 정원은 IMF시절에 혹독한 구조조정을 거치면서 2002년에는 788명으로 200여명이 가까이 줄어들었으나 이번 정원 확보로 공무원 정원 1000명 시대를 맞게 됐다. 
그동안 광양시는 포스코를 중심으로 한 산업단지와 컨테이너부두, 경제자유구역, 택지 개발, 다양한 복지 정책 등 시민의 삶의 질 향상을 위한 행정수요가 날로 증폭하는 데에 비해 한정된 공무원 정원으로 많은 어려움을 겪어 왔었다. 
시의 이번 정원 충원은 지금까지 격무에 시달려온 공무원들에게 더욱 열심히 일할 수 있도록 활력을 불어넣으면서 지역과 나라 발전을 뒷받침 하는 경제도시로서의 위상을 드높이는 시너지 효과로 이어질 것으로 기대된다. 
한편, 광양시는 한시기구인‘기업유치추진단’이 2017년 연말로 폐지됨에 따라 전라남도로부터 4급 상설기구인‘산단녹지관리센터’를 승인 받아 올해 1월부터 운영하게 됨으로써 도시 개발에 한층 탄력을 받을 것으로 전망하고 있다. 
정홍기 총무과장은“참으로 어렵게 가져온 공무원 정원과 4급 상설기구는 격무에 시달려온 직원들의 숨통을 트이게 해 15만 시민을 위한 행정서비스 질을 향상시킬 것으로 기대된다”며“이달 안에 정원 조례를 개정하고, 직무분석을 통해 부서별 정원을 조정할 계획이다”고 말했다.

한국어 3문장으로 위의 Document를 요약해줘.
### Assistant: 1)"""
encodings = tokenizer(text, max_length=2048, padding="max_length", truncation=True, return_tensors="np")

In [ ]:
temperature = 0.8
max_length = 1024

@pjit
def generate(x: chex.Array, mask: chex.Array, params: chex.ArrayTree, rng: chex.PRNGKey) -> chex.Array:
    rng, new_rng = jax.random.split(rng)
    generated = jnp.zeros((x.shape[0], max_length), dtype=jnp.int32)

    logits, variables = model.apply({"params": params}, x, mask, mutable=["cache"])
    new_tokens = jax.random.categorical(rng, logits[:, -1, :] / temperature)
    generated = jnp.roll(generated, -1, 1).at[:, -1].set(new_tokens)
    
    def body_fn(_: int, state: tuple[chex.Array, ...]):
        x, cache, rng, generated = state
        rng, new_rng = jax.random.split(rng)

        logits, variables = model.apply({"params": params, "cache": cache}, x[:, None], mutable=["cache"])
        new_tokens = jax.random.categorical(rng, logits[:, -1, :] / temperature)
        generated = jnp.roll(generated, -1, 1).at[:, -1].set(new_tokens)
        return new_tokens, variables["cache"], new_rng, generated
    
    state = (new_tokens, variables["cache"], new_rng, generated)
    state = jax.lax.fori_loop(0, max_length - 1, body_fn, init_val=state)
    return state[3]

In [ ]:
with mesh:
    generated = generate(
        jnp.asarray(encodings.input_ids, dtype=jnp.int32),
        jnp.asarray(encodings.attention_mask, dtype=jnp.bool_),
        params,
        jax.random.PRNGKey(76),
    )
    generated.block_until_ready()

In [ ]:
print(tokenizer.decode(generated[0].tolist()).split("###")[0])